In [2]:
import json
import numpy as np
import os
from collections import *
from tqdm import tqdm
from scipy.sparse import csr_matrix, vstack, csr_array
import time
import pickle, gzip, joblib, shelve
import tkinter as tk
from tkinter import ttk, font
import time
from itertools import islice, combinations
from datetime import datetime

In [33]:
directory_path = 'data/raw'

In [34]:
fullpaths = [directory_path + '/' + f for f in filenames][995:]
print(len(fullpaths) * 1000)
save = False

5000


In [18]:
with open('/Users/idotzhori/Desktop/Linear Algebra/SongRank2/data_test/challenge_set.json') as f:
    mpd_slice = json.load(f)

playlists = mpd_slice['playlists']

In [35]:
fullpaths

['data/raw/mpd.slice.995000-995999.json',
 'data/raw/mpd.slice.996000-996999.json',
 'data/raw/mpd.slice.997000-997999.json',
 'data/raw/mpd.slice.998000-998999.json',
 'data/raw/mpd.slice.999000-999999.json']

In [14]:
# to load cooccurrence_matrix from storage
f_time = '01_12_08_2023'
save_path = os.path.join('song_data', f'{f_time}_cooccurrence_matrix.gz')

with gzip.open(save_path, 'rb') as f:
    cooccurrence_matrix = pickle.load(f)

In [5]:
folder_path = 'song_data/pmi'
saved_chunk_files = [folder_path + '/' + f for f in os.listdir(folder_path) if "01_12_08_2023" in f]
saved_chunk_files = sorted(saved_chunk_files, key = lambda x: int(x.split('_')[5]))
chunks = []
for chunk_file in saved_chunk_files:
    with gzip.open(chunk_file, 'rb') as f:
        chunks.append(pickle.load(f))

pmi_matrix = vstack(chunks, format='csr')
print(f'Finished processing matrix size: {pmi_matrix.shape}')

Finished processing matrix size: (1341936, 1341936)


In [6]:
f_time = '00_12_08_2023'
save_path = os.path.join('song_data', f'{f_time}_song_indices.gz')

with gzip.open(save_path, 'rb') as f:
    song_indices = pickle.load(f)

num_songs = len(song_indices)
print(num_songs)

1341936


In [9]:
def user_playlist_vector(playlist_songs, song_indices, num_songs):
    if len(playlist_songs) == 0:
        user_vector = np.ones(num_songs)
    user_vector = np.zeros(num_songs)
    for song in playlist_songs:
        user_vector[song_indices[song]] = 1
    return csr_array(user_vector)

def compute_scores(user_vector, pmi_matrix):
    scores = user_vector.dot(pmi_matrix)
    return scores.toarray()[0]

def get_top_recommendations(scores, song_data_map, n=10):
    top_indices = np.argsort(scores)[-n:]
    top_songs = [song_data_map[list(song_indices)[i]] for i in top_indices]
    return top_songs

def recommend_songs_pmi(user_playlist, song_indices, pmi_matrix, song_data_map, n=10):
    user_vector = user_playlist_vector(user_playlist, song_indices, num_songs)
    scores = compute_scores(user_vector, pmi_matrix)
    return get_top_recommendations(scores, song_data_map, n)